In [1]:
import torch
import torch.nn as nn
import numpy as np
import os
import sys
sys.path.append("/home/tigeriv/Code/NuScenesConvBKI/EndToEnd/")
from Segmentation.spvcnn import *
from torchsparse import SparseTensor
from torchsparse.utils.quantize import sparse_quantize
from torchsparse.utils.collate import sparse_collate

In [2]:
from nuscenes.nuscenes import NuScenes
from nuscenes.utils.data_classes import LidarPointCloud
from nuscenes.utils.data_classes import LidarSegPointCloud
from nuscenes.utils.data_io import load_bin_file
from pyquaternion import Quaternion
from nuscenes.utils.geometry_utils import transform_matrix
data_dir = '/home/tigeriv/Data/nuscenes_mini'
nusc = NuScenes(version='v1.0-mini', dataroot=data_dir, verbose=True)

AssertionError: Database version not found: /home/tigeriv/Data/nuscenes_mini/v1.0-mini

In [10]:
dev = 'cuda' if torch.cuda.is_available() else 'cpu'

dtype=torch.float32

voxel_sizes = torch.tensor([0.25, 0.25, 0.25]).to(dev)

grid_size = torch.tensor([200, 200, 20]).to(dev)
min_bound = torch.tensor([-25., -25., -3.5]).to(dev)
max_bound = torch.tensor([25., 25., 1.5]).to(dev)

# num_classes = 13
num_classes = 19

f = 5

# seg_path = "/home/tigeriv/Code/NuScenesConvBKI/spvnas/runs/Downsampled/checkpoints/max-iou-test.pt"
seg_path = "/home/tigeriv/Code/NuScenesConvBKI/spvnas/runs/KITTI/checkpoints/max-iou-test.pt"

In [11]:
seg_net = SPVCNN(
    num_classes=num_classes,
    cr=0.5,
    pres=0.05,
    vres=0.05).to(dev)

# seg_net.load_state_dict(torch.load(seg_path)['model'])
seg_net.load_state_dict(torch.load(seg_path))

<All keys matched successfully>

In [12]:
print(seg_net)

SPVCNN(
  (stem): Sequential(
    (0): Conv3d(4, 16, kernel_size=(3, 3, 3), bias=False)
    (1): BatchNorm(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv3d(16, 16, kernel_size=(3, 3, 3), bias=False)
    (4): BatchNorm(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
  )
  (stage1): Sequential(
    (0): BasicConvolutionBlock(
      (net): Sequential(
        (0): Conv3d(16, 16, kernel_size=(2, 2, 2), stride=(2, 2, 2), bias=False)
        (1): BatchNorm(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
    )
    (1): ResidualBlock(
      (net): Sequential(
        (0): Conv3d(16, 16, kernel_size=(3, 3, 3), bias=False)
        (1): BatchNorm(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv3d(16, 16, kernel_size=(3, 3, 3), bias=False)
        (4): BatchNorm

In [5]:
def get_lidar(curr_sample):
    # Load lidar pc
    lidar_data = nusc.get('sample_data', curr_sample['data']['LIDAR_TOP'])
    ego_pose = nusc.get('ego_pose', lidar_data['ego_pose_token'])
    lidar_pose = transform_matrix(translation=ego_pose['translation'], rotation=Quaternion(ego_pose['rotation']))
    
    lidar_fpath = os.path.join(data_dir, lidar_data['filename'])
    label_data = nusc.get('lidarseg', curr_sample['data']['LIDAR_TOP'])
    label_fpath = os.path.join(data_dir, label_data['filename'])

    labeled_pc = LidarSegPointCloud(points_path=lidar_fpath, labels_path=label_fpath)
    return lidar_pose, labeled_pc
    

def generate_seg_in(lidar):
    # Create input data
    coords = np.round(lidar[:, :3] / 0.05)
    coords -= coords.min(0, keepdims=1)
    feats = lidar
    # Filter out duplicate points
    coords, indices, inverse = sparse_quantize(coords, return_index=True, return_inverse=True)
    coords = torch.tensor(coords, dtype=torch.int)
    feats = torch.tensor(feats[indices], dtype=torch.float)

    inputs = SparseTensor(coords=coords, feats=feats)
    inputs = sparse_collate([inputs]).cuda()
    return inputs, inverse

In [6]:
curr_scene = nusc.scene[2]

sample_token = curr_scene['first_sample_token']
curr_sample = nusc.get('sample', sample_token)

In [11]:
curr_sample = nusc.get('sample', curr_sample['next'])

lidar_pose, labeled_pc = get_lidar(curr_sample)
lidar = labeled_pc.points
print(lidar.shape)
seg_input, inv = generate_seg_in(lidar)

(34720, 4)


In [8]:
# Return features
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x

seg_net.classifier = Identity()

In [9]:
labeled_points = seg_net(seg_input)[inv]
print(labeled_points.shape)

torch.Size([34688, 48])


In [10]:
print(seg_net)

SPVCNN(
  (stem): Sequential(
    (0): Conv3d(4, 16, kernel_size=(3, 3, 3), bias=False)
    (1): BatchNorm(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv3d(16, 16, kernel_size=(3, 3, 3), bias=False)
    (4): BatchNorm(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
  )
  (stage1): Sequential(
    (0): BasicConvolutionBlock(
      (net): Sequential(
        (0): Conv3d(16, 16, kernel_size=(2, 2, 2), stride=(2, 2, 2), bias=False)
        (1): BatchNorm(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
      )
    )
    (1): ResidualBlock(
      (net): Sequential(
        (0): Conv3d(16, 16, kernel_size=(3, 3, 3), bias=False)
        (1): BatchNorm(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU(inplace=True)
        (3): Conv3d(16, 16, kernel_size=(3, 3, 3), bias=False)
        (4): BatchNorm